In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, count


In [2]:
spark = SparkSession.builder.master('local[*]').appName('used-car-price').config('spark.executor.memory', '6g').getOrCreate()

24/09/10 17:56:39 WARN Utils: Your hostname, langchain resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp3s0)
24/09/10 17:56:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/10 17:56:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/10 17:56:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/10 17:56:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/09/10 17:56:40 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
spark = SparkSession.builder.master('local[*]').appName('used-car-price')\
.config("spark.executor.instances", "1")\
.config("spark.executor.memory", "6g")\
.config("spark.driver.memory", "6g")\
.config("spark.executor.memoryOverhead", "8g")\
.getOrCreate()

24/09/10 17:56:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
csv_file_path = '../data/test.csv'
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

In [5]:
df.show()

+------+-------------+--------------------+----------+------+---------+--------------------+--------------------+--------------------+---------+--------------------+-----------+
|    id|        brand|               model|model_year|milage|fuel_type|              engine|        transmission|             ext_col|  int_col|            accident|clean_title|
+------+-------------+--------------------+----------+------+---------+--------------------+--------------------+--------------------+---------+--------------------+-----------+
|188533|         Land|      Rover LR2 Base|      2015| 98000| Gasoline|240.0HP 2.0L 4 Cy...|         6-Speed A/T|               White|    Beige|       None reported|        Yes|
|188534|         Land|   Rover Defender SE|      2020|  9142|   Hybrid|395.0HP 3.0L Stra...|         8-Speed A/T|              Silver|    Black|       None reported|        Yes|
|188535|         Ford|  Expedition Limited|      2022| 28121| Gasoline|3.5L V6 24V PDI D...|  10-Speed Automat

In [6]:
# from pyspark.sql.functions import when

# # List of rare engine types when analysing train data
# rare_engines = [
#     "70.0HP 1.0L 3 Cylinder Engine Gasoline Fuel",
#     "443.0HP 4.4L 8 Cylinder Engine Gasoline Fuel",
#     "111.0HP Electric Motor Electric Fuel System",
#     "184.0HP 2.4L 4 Cylinder Engine Flex Fuel Capability",
#     "151.0HP 1.5L 4 Cylinder Engine Gas/Electric Hybrid",
#     "139.0HP 1.6L 4 Cylinder Engine Plug-In Electric/Gas",
#     "78.0HP 1.2L 3 Cylinder Engine Gasoline Fuel",
#     "313.0HP 2.0L 4 Cylinder Engine Plug-In Electric/Gas"
# ]

# df = df.withColumn('engine', when(col('engine').isin(rare_engines), 'Rare_engine').otherwise(col('engine')))


# df.show()


In [7]:
# engine_counts = df.groupBy('engine').agg(count('engine').alias('count')).orderBy('count') 
# engine_counts.show(n=52)

In [8]:
df

DataFrame[id: int, brand: string, model: string, model_year: int, milage: int, fuel_type: string, engine: string, transmission: string, ext_col: string, int_col: string, accident: string, clean_title: string]

In [9]:
from pyspark.sql.functions import col, when,regexp_replace,lower
df = df.withColumn(
    "transmission",
    when(col("transmission").contains("At/Mt") | col("transmission").contains("AT/MT"), 
          regexp_replace(col("transmission"), "AT/MT|At/Mt", "automatic/manual"))
    .when(col("transmission").contains("A/T") | col("transmission").contains("AT"), 
         regexp_replace(col("transmission"), "A/T|AT", "automatic"))
    .when(col("transmission").contains("Mt") | col("transmission").contains("M/T"), 
          regexp_replace(col("transmission"), "Mt|M/T", "manual"))
    
    .when(col("transmission").contains("F"), "single-speed fixed gear")
    .when(col("transmission").contains("DCT"), "dual clutch automatic")
    .when(col("transmission").contains("CVT"), "continuously variable transmission")
    .when(col("transmission").contains("5-Speed M/T"), "5-Speed manual")
    .otherwise(col("transmission"))
)
df = df.withColumn("transmission", lower(col("transmission")))

In [10]:
transmission_counts = df.groupBy('transmission').agg(count('transmission').alias('count')).orderBy('count')
transmission_counts.show(n=52)

+--------------------+-----+
|        transmission|count|
+--------------------+-----+
|dual clutch autom...|   11|
|6 speed automatic...|   14|
|             6-speed|   14|
|             7-speed|   21|
|      8-speed manual|   23|
|9-speed automatic...|   28|
|6-speed electroni...|   29|
|      6 speed manual|   29|
|10-speed automati...|   39|
|                   –|   40|
|            variable|   58|
|   2-speed automatic|   58|
|6-speed automatic...|   62|
|                   2|   64|
|single-speed fixe...|   75|
|transmission over...|  124|
|8-speed automatic...|  345|
|      7-speed manual|  369|
|              manual|  759|
|7-speed automatic...|  800|
|   1-speed automatic| 1526|
|      5-speed manual| 1639|
|   4-speed automatic| 1701|
|continuously vari...| 1878|
|   5-speed automatic| 2372|
|   9-speed automatic| 4137|
|  10-speed automatic| 7407|
|   7-speed automatic| 7486|
|      6-speed manual| 7995|
|transmission w/du...|12824|
|   6-speed automatic|13952|
|   8-speed au

In [12]:
df = df.withColumn(
    "transmission",
    when(col("transmission") == "–", "unknown-transmission")
    .otherwise(col("transmission"))
)

df.select('transmission').distinct().show()

+--------------------+
|        transmission|
+--------------------+
|6-speed electroni...|
|6 speed automatic...|
|   5-speed automatic|
|7-speed automatic...|
|            variable|
|  10-speed automatic|
|   2-speed automatic|
|           automatic|
|             7-speed|
|10-speed automati...|
|      7-speed manual|
|single-speed fixe...|
|   6-speed automatic|
|transmission over...|
|9-speed automatic...|
|   8-speed automatic|
|              manual|
|      5-speed manual|
|      6 speed manual|
|      8-speed manual|
+--------------------+
only showing top 20 rows



In [14]:
from pyspark.sql import functions as F
df.groupBy("transmission").agg(F.count("*").alias("count")).orderBy("count", ascending=False).show(truncate=False)

+----------------------------------+-----+
|transmission                      |count|
+----------------------------------+-----+
|automatic                         |40307|
|8-speed automatic                 |19504|
|6-speed automatic                 |13952|
|transmission w/dual shift mode    |12824|
|6-speed manual                    |7995 |
|7-speed automatic                 |7486 |
|10-speed automatic                |7407 |
|9-speed automatic                 |4137 |
|5-speed automatic                 |2372 |
|continuously variable transmission|1878 |
|4-speed automatic                 |1701 |
|5-speed manual                    |1639 |
|1-speed automatic                 |1526 |
|7-speed automatic with auto-shift |800  |
|manual                            |759  |
|7-speed manual                    |369  |
|8-speed automatic with auto-shift |345  |
|transmission overdrive switch     |124  |
|single-speed fixed gear           |75   |
|2                                 |64   |
+----------

In [15]:

df = df.withColumn(
    "accident",
    when(col("accident") == "At least 1 accident or damage reported", "damage reported")
    .otherwise(col("accident"))
)
df.select('accident').distinct().show()

+---------------+
|       accident|
+---------------+
|  None reported|
|damage reported|
|           NULL|
+---------------+



In [16]:
df = df.withColumn(
    "fuel_type",
    when(col("fuel_type") == "–", "unknown-name-fuel")
    .otherwise(col("fuel_type"))
)

df.select('fuel_type').distinct().show()

+-----------------+
|        fuel_type|
+-----------------+
|         Gasoline|
|    E85 Flex Fuel|
|           Diesel|
|           Hybrid|
|   Plug-In Hybrid|
|unknown-name-fuel|
|    not supported|
|             NULL|
+-----------------+



In [17]:
def remove_outliers_iqr(df, column):

    Q1 = df.approxQuantile(column, [0.25], 0.01)[0]
    Q3 = df.approxQuantile(column, [0.75], 0.01)[0]
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


    df_out = df.filter((F.col(column) >= lower_bound) & (F.col(column) <= upper_bound))
    return df_out

df_train_no_outliers = remove_outliers_iqr(df, 'milage')

In [18]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re
from pyspark.sql import functions as F

def clean_engine(engine):
    engine = re.sub(r"[^a-zA-Z0-9\s]", "", engine)
    return engine

clean_engine_udf = udf(clean_engine, StringType())
df = df.withColumn("engine", clean_engine_udf(F.col("engine")))


In [19]:
output_path = "../data/cleaned_test.csv"
df_train_no_outliers.write.csv(output_path, header=True, mode='overwrite')
